In [1]:
from utils import SurrogatPyRepEnvironment
from basic_walk.utils import BaseAgent
import sys
import time

from multiprocessing import Pool
import pickle
import matplotlib.pyplot as plt
import numpy as np

from tqc import structures, DEVICE
from tqc.trainer import Trainer
from tqc.structures import Actor, Critic, RescaleAction
from tqc.functions import eval_policy
from tqdm import tqdm
import copy

import warnings
warnings.filterwarnings('ignore')

## Заполняем буффер реплеев

In [2]:
def generate_correct_replays(name, goal_len, episode_length, seed=42):
    print(f"{name} started with goal_len: {goal_len}")
    np.random.seed(seed)
    agent = BaseAgent(random_mode=True, foot_only_mode=True)
    replay_buffer = []
    with SurrogatPyRepEnvironment('scenes/basic_scene.ttt', headless_mode=True, foot_only_mode=True) as env:
        env = RescaleAction(env, -1., 1.)
        state, done = env.reset(), False
        episode_timesteps = 0
        last_replay = []
        percentage_to_achive = 0.05

        while len(replay_buffer) < goal_len:
            episode_timesteps += 1
            action = agent.act(state)
            action = np.array(action)
            action /= np.pi

            next_state, reward, done, _ = env.step(action)
            last_replay.append((state, action, next_state, reward, done))
            state = next_state
            
            
            if done or episode_timesteps >= episode_length:
                if not done:
                    replay_buffer += last_replay
#                     print(f"{name}: {len(replay_buffer)} ({float(len(replay_buffer)) / goal_len})")
                    if float(len(replay_buffer)) / goal_len >= percentage_to_achive:
                        print(f"{name}: {int(percentage_to_achive*100)}%")
                        percentage_to_achive += 0.05
                    
                agent = BaseAgent(random_mode=True, foot_only_mode=True)
                state, done = env.reset(), False
                episode_timesteps = 0
                last_replay = []
            

    return replay_buffer

In [3]:
def generate_correct_replays_in_parallel(goal_len=1200, episode_length=600, n_proc=2, seed=42):
    np.random.seed(seed)
    rng = np.random.default_rng()
    seeds = rng.choice(100000, size=n_proc, replace=False)
    
    assert goal_len % (episode_length * n_proc) == 0
    sub_process_goal_len = goal_len / n_proc
    replay_buffer = []
    
    with Pool(processes=n_proc) as pool:
        multiple_results = []
        for i, seed in enumerate(seeds):
            res = pool.apply_async(generate_correct_replays, (i, sub_process_goal_len, episode_length, seed))
            multiple_results.append(res)
        for i, res in enumerate(multiple_results):
            res.wait()
            print("finish", i)

        for res in multiple_results:
            subreplay = res.get()
            replay_buffer += subreplay
    return replay_buffer

In [4]:
%%time
goal_len = 200000
# goal_len = 40000
episode_length = 100
seed = 42

replay_buffer_arr = generate_correct_replays_in_parallel(
    goal_len=goal_len,
    episode_length=episode_length,
    seed=seed,
    n_proc=8
)
buffer_name = "replay_buffer_leg_only"

file_name = f"data/replay_buffers/{buffer_name}_array_{goal_len}_{episode_length}.pickle"
with open(file_name, 'wb') as f:
    pickle.dump(replay_buffer_arr, f)

with SurrogatPyRepEnvironment('scenes/basic_scene.ttt', headless_mode=True, foot_only_mode=True) as env:
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.shape[0]
    
    replay_buffer = structures.ReplayBuffer(state_dim, action_dim)
    
    for q in replay_buffer_arr:
        replay_buffer.add(*q)

    file_name = f"data/replay_buffers/{buffer_name}_{goal_len}_{episode_length}.pickle"
    with open(file_name, 'wb') as f:
        pickle.dump(replay_buffer, f)

0 started with goal_len: 25000.0
1 started with goal_len: 25000.0
3 started with goal_len: 25000.0
2 started with goal_len: 25000.0
6 started with goal_len: 25000.0
4 started with goal_len: 25000.07 started with goal_len: 25000.05 started with goal_len: 25000.0


7: 5%
4: 5%
6: 5%
0: 5%
5: 5%
2: 5%
3: 5%
1: 5%
4: 10%
7: 10%
3: 10%
6: 10%
2: 10%
1: 10%
0: 10%
5: 10%
3: 15%
7: 15%
6: 15%
4: 15%
1: 15%
5: 15%
2: 15%
0: 15%
7: 20%
4: 20%
5: 20%
1: 20%
6: 20%
3: 20%
2: 20%
0: 20%
7: 25%
4: 25%
6: 25%
2: 25%
1: 25%
3: 25%
0: 25%
7: 30%
5: 25%
6: 30%
4: 30%
1: 30%
2: 30%
3: 30%
0: 30%
7: 35%
5: 30%
1: 35%
4: 35%
6: 35%
2: 35%
7: 40%
5: 35%
0: 35%
3: 35%
4: 40%
2: 40%
6: 40%
1: 40%
7: 44%
5: 40%
0: 40%
4: 44%
3: 40%
2: 44%
1: 44%
6: 44%
7: 49%
5: 44%
3: 44%
2: 49%
0: 44%
1: 49%
4: 49%
6: 49%
7: 54%
5: 49%
0: 49%
3: 49%
6: 54%
4: 54%
2: 54%
7: 60%
1: 54%
3: 54%
5: 54%
6: 60%
0: 54%
2: 60%
4: 60%
1: 60%
7: 65%
3: 60%
6: 65%
5: 60%
0: 60%
7: 70%
2: 65%
4: 65%
1: 65%
3: 65%
0: 65%
6: 70%
5: 65%
7:

In [6]:
replay_buffer.size

160000